In [ ]:
install.packages("Mcomp")
install.packages("forecast")
install.packages("data.table")
install.packages("magrittr")
install.packages("stringr")
install.packages("foreach")

In [ ]:
library(Mcomp)
library(forecast)
packageVersion("forecast")
library(data.table)
library(magrittr)
library(stringr)
library(foreach)

In [ ]:
M4M_actuals_train <- fread('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Monthly-train.csv', header = TRUE, fill = TRUE)
M4M_actuals_test <- fread('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Monthly-test.csv', header = TRUE, fill = TRUE)

setnames(M4M_actuals_train, names(M4M_actuals_train), c('item_id', 1:2794))
setnames(M4M_actuals_test, names(M4M_actuals_test), c('item_id', 1:18))
M4M_actuals_train[, item_id := str_remove(item_id, 'M') %>% as.numeric()]
M4M_actuals_test[, item_id := str_remove(item_id, 'M') %>% as.numeric()]

M4M_actuals_all <- merge.data.table(M4M_actuals_train, M4M_actuals_test, by = 'item_id')
setnames(M4M_actuals_all, names(M4M_actuals_all), c('item_id', 1:2812))
M4M_actuals_all <- melt.data.table(M4M_actuals_all, id.vars = 'item_id')
M4M_actuals_all <- M4M_actuals_all[!is.na(value)]
setorder(M4M_actuals_all, variable)
M4M_actuals_all[, variable:= c(1:.N) , by = list(item_id)]

In [ ]:
start_item <- 1
end_item <- 10

In [ ]:
M4M_actuals_all_df <- data.frame(M4M_actuals_all)

starttime <- Sys.time()

ets_forecasts_df <- foreach(id = c(start_item:end_item), .combine = 'rbind') %do% {

  ets_forecasts_mean_df <- data.frame()
  ets_forecasts_sd_df <- data.frame()
  ets_insample_sd_df <- data.frame()

  ts <- M4M_actuals_all_df[which(M4M_actuals_all_df$item_id == id),]$value

    counter <- 1
    for (fc_origin in 1:13){

      ts_origin <- ts(ts[1:(length(ts)-(18-(fc_origin-1)))], frequency = 12)

      ets_fit <- ets(ts_origin)
      ets_forecasts <- forecast(ets_fit, h = 6, PI = TRUE, level = c(0.9))

      ets_forecasts_mean_df[counter, 1] <- id
      ets_forecasts_mean_df[counter, 2] <- fc_origin
      ets_forecasts_mean_df[counter, c(3:8)] <- ets_forecasts$mean
      ets_forecasts_mean_df[counter, 9] <- 'mean_forecast'

      ets_forecasts_sd_df[counter, 1] <- id
      ets_forecasts_sd_df[counter, 2] <- fc_origin
      #ets_forecasts_sd_df[counter, c(3:8)] <- (ets_forecasts$upper - ets_forecasts$mean)/qnorm(0.9)
      ets_forecasts_sd_df[counter, c(3:8)] <- (ets_forecasts$upper - ets_forecasts$mean)/qnorm(0.95)
      ets_forecasts_sd_df[counter, 9] <- 'sd_forecast'

      ets_insample_sd_df[counter, 1] <- id
      ets_insample_sd_df[counter, 2] <- fc_origin
      ets_insample_sd_df[counter, c(3:8)] <- rep(accuracy(ets_fit)[2], 6)
      ets_insample_sd_df[counter, 9] <- 'sd_insample'

      counter <- counter+1
    }

    #output
    rbind(ets_forecasts_mean_df, ets_forecasts_sd_df, ets_insample_sd_df)

}

Sys.time() - starttime

In [ ]:
colnames(ets_forecasts_df) <- c('item_id', 'fc_origin', as.character(1:6), 'type')
ets_forecasts_dt <- ets_forecasts_df %>% data.table()


In [ ]:
ets_forecasts_dt